In [2]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run[0:run.rfind('_')]


def get_dataset_size(dataset):
    if dataset == "osm" or dataset == "books":
        return 800_000_000
    else:
        return 200_000_000

runs = []

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/eps_study'
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results' )):
        if run_name == 'run':
            continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)

df = pd.json_normalize(runs)



print(df.columns)

Index(['command', 'dataset', 'dataset_size', 'result.checksum',
       'result.duration_ns', 'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


In [3]:
df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["tput"] = (df["dataset_size"] / df["ratio"]) / (df["result.duration_ns"] / (1000000000))
df["algo"] = df["spec.algo_name"]
display(df["algo"].unique())
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["epsilon"] = df["spec.index.epsilon"]


array(['lsj_sampledflatpgm256', 'hash_join', 'sort_join', 'inlj_btree256',
       'lsj_btree256', 'inlj_sampledflatpgm256'], dtype=object)

In [6]:
rows = duckdb.sql("SELECT ratio, dataset, algo, avg(tput) as v FROM df GROUP BY ratio, dataset,algo ORDER BY ratio").df()
rows.pivot(index='ratio', values='v', columns=['dataset', 'algo'])


dataset            fb                                                    \
algo    inlj_btree256     hash_join     sort_join lsj_sampledflatpgm256   
ratio                                                                     
1        1.279714e+07  1.043630e+07  3.362000e+07          4.051299e+07   
10       9.275944e+06  2.273716e+06  1.406034e+07          2.500512e+07   
100      3.817939e+06  3.341801e+05  2.721962e+06          7.178850e+06   
1000     1.292361e+06  6.756465e+04  2.864227e+05          1.626335e+06   

dataset     lognormal uniform_sparse                     fb  \
algo     lsj_btree256   lsj_btree256 inlj_sampledflatpgm256   
ratio                                                         
1        4.881760e+06   4.890289e+06           1.794241e+07   
10       3.600536e+06   3.673006e+06           1.103762e+07   
100      1.667520e+06   1.580638e+06           3.633578e+06   
1000     8.787592e+05   8.778421e+05           1.112156e+06   

dataset        uniform_sparse            fb     lognormal                \
algo    lsj_sampledflatpgm256  lsj_btree256     hash_join     sort_join   
ratio                                                                     
1                9.491795e+06  1.709012e+07  2.521930e+06  1.759690e+07   
10                        NaN  1.145245e+07  4.147267e+05  5.353341e+06   
100                       NaN  4.816411e+06  5.691920e+04  7.340373e+05   
1000                      NaN  1.422218e+06  1.429337e+04  6.919465e+04   

dataset                       uniform_sparse                 
algo    lsj_sampledflatpgm256      hash_join      sort_join  
ratio                                                        
1                9.340279e+06            NaN            NaN  
10               6.172101e+06  433209.919792            NaN  
100              2.280978e+06   62132.705617  732581.507032  
1000             7.009279e+05   15310.350668            NaN

In [4]:
# Ratio = 1, epsilon vs duration across multiple threads
def plot_for_ratio(dataset):
    rows = duckdb.sql(
        " SELECT dataset, threads, ratio, epsilon, algo, AVG(duration_sec) as v FROM df"
        " WHERE (algo='lsj_flatpgm256' OR algo='lsj_flatpgm1024' OR algo='lsj_flatpgm4096')"
        f" AND dataset='{dataset}'"
        " GROUP BY dataset, ratio, threads, epsilon, algo"
        " ORDER BY ratio, threads, epsilon"
    ).df()
    return rows.pivot(index='threads', values='v', columns=['epsilon', 'ratio'])


In [5]:
# Ratio = 1, epsilon vs duration across multiple threads
display('fb', plot_for_ratio('fb'))
display('wiki', plot_for_ratio('wiki'))
display('osm', plot_for_ratio('osm'))
display('books', plot_for_ratio('books'))
display('udense', plot_for_ratio('uniform_dense'))
display('usparse', plot_for_ratio('uniform_sparse'))
display('normal', plot_for_ratio('normal'))
display('lognormal', plot_for_ratio('lognormal'))
display('string_keys', plot_for_ratio('string_keys'))

'fb'

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,26.190664,28.362248,30.211240,4.088599,4.086847,4.564430,1.104157,1.045334,0.977828,0.299930,0.462068,0.495449
2,13.609405,14.452971,17.613453,2.065043,2.133508,2.490935,0.569503,0.521568,0.529227,0.173093,0.235491,0.267236
4,8.389189,8.784014,9.723148,1.468388,1.512390,1.483382,0.325730,0.298773,0.327492,0.089801,0.131088,0.155002
8,5.996413,6.155027,6.441893,0.888175,0.922249,0.978765,0.219654,0.220341,0.220129,0.096747,0.119502,0.122903
16,4.995718,5.025974,4.683901,0.645955,0.666504,0.706084,0.210582,0.218051,0.229434,0.175474,0.142004,0.144517


'wiki'

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,11.596428,12.720760,14.053581,3.341000,3.319841,3.565870,0.679447,0.743503,0.636850,0.216608,0.271517,0.283918
2,6.697459,6.743496,7.464118,2.122817,1.608226,1.976445,0.380171,0.342897,0.420474,0.131939,0.149033,0.158578
4,3.925824,4.075155,4.534856,1.125969,1.034591,1.362840,0.237696,0.247868,0.251605,0.078038,0.095496,0.110595
8,3.088491,2.884251,3.676284,0.751399,0.799041,0.861033,0.189785,0.180700,0.193569,0.086375,0.111525,0.096064
16,2.323467,2.393951,2.508316,0.596770,0.581947,0.596662,0.160427,0.174208,0.175645,0.110819,0.114311,0.128340


'osm'

...

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,26.234829,28.096915,30.368441,3.561598,4.205500,4.586375,0.994933,0.785136,1.005801,0.289290,0.412675,0.438096
2,13.330235,14.845427,16.284912,1.874656,2.134439,2.419884,0.475625,0.427696,0.481612,0.147392,0.202019,0.240564
4,8.091567,8.874047,9.724246,1.470205,1.545345,1.665798,0.305228,0.304426,0.294460,0.091512,0.118788,0.143066
8,6.088394,6.344533,6.169768,0.837402,0.937439,1.002453,0.214783,0.216014,0.215562,0.090225,0.105144,0.119148
16,5.033523,5.309408,5.241919,0.640698,0.685556,0.698924,0.222017,0.189023,0.196822,0.138839,0.127609,0.142594


'lognormal'

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,24.927506,27.321879,29.471769,4.003646,4.365843,4.476430,1.002268,0.878553,0.984343,0.306410,0.390009,0.435149
2,12.715500,14.125942,15.090289,2.161790,2.321908,2.490459,0.516857,0.545615,0.515337,0.152004,0.194671,0.261948
4,8.066458,8.365460,9.157828,1.417272,1.356959,1.633166,0.310736,0.302459,0.308466,0.091847,0.124318,0.148240
8,5.502039,6.286209,6.376248,0.891186,0.912284,0.955612,0.209971,0.210362,0.253809,0.109578,0.111108,0.121531
16,4.701473,4.787259,5.263620,0.650854,0.676181,0.687381,0.198089,0.194152,0.199078,0.107273,0.151194,0.144077


'string_keys'

epsilon,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0,256.0,1024.0,4096.0
ratio,1,1,1,10,10,10,100,100,100,1000,1000,1000
threads,,,,,,,,,,,,
1,11.664905,12.913584,14.115216,2.414677,2.668440,2.996465,0.509996,0.500652,0.631432,0.172064,0.220596,0.224482
2,6.295998,6.602873,7.399803,1.553230,1.626821,1.707614,0.269913,0.299512,0.341678,0.086560,0.118055,0.133571
4,4.125174,4.000628,4.666980,0.995457,0.885024,1.142465,0.168566,0.198207,0.180611,0.056449,0.080864,0.077870
8,3.157444,2.803570,3.300959,0.631978,0.733862,0.734320,0.146072,0.150710,0.156084,0.074831,0.072669,0.085563
16,2.659379,2.761355,2.728846,0.576156,0.582016,0.550030,0.175648,0.156276,0.180392,0.114147,0.123222,0.104528


In [6]:

def plot_epsilon_vs_duration_200M(ratio):
    rows = duckdb.sql(
        " SELECT threads, ratio, epsilon, algo, AVG(duration_sec) as v FROM df"
        " WHERE (algo='lsj_flatpgm256' OR algo='lsj_flatpgm1024' OR algo='lsj_flatpgm4096')"
        f" AND ratio={ratio} AND dataset_size=200000000"
        " GROUP BY ratio, threads, epsilon, algo"
        " ORDER BY ratio, threads, epsilon"
    ).df()
    os.makedirs('epsilon_vs_duration_csv', exist_ok=True)
    rows.pivot(index='epsilon', values='v', columns=['threads']).to_csv(f'epsilon_vs_duration_csv/ratio_{ratio}.txt')
    return rows.pivot(index='epsilon', values='v', columns=['threads'])

display(plot_epsilon_vs_duration_200M(1))
display(plot_epsilon_vs_duration_200M(10))
display(plot_epsilon_vs_duration_200M(100))
display(plot_epsilon_vs_duration_200M(1000))

threads,1,2,4,8,16
epsilon,,,,,
256.0,21.916101,11.504381,7.137959,5.148922,4.208252
1024.0,23.867351,12.359299,7.451095,5.265870,4.278030
4096.0,25.783860,13.767512,8.122447,5.662545,4.429822


threads,1,2,4,8,16
epsilon,,,,,
256.0,3.598429,2.019615,1.358089,0.844028,0.632011
1024.0,3.872437,2.024734,1.327048,0.888148,0.652870
4096.0,4.161348,2.218484,1.522538,0.937902,0.665862


threads,1,2,4,8,16
epsilon,,,,,
256.0,0.876144,0.468826,0.285175,0.200511,0.195171
1024.0,0.852496,0.447647,0.281473,0.199689,0.191541
4096.0,0.905022,0.473725,0.289584,0.214790,0.195265


threads,1,2,4,8,16
epsilon,,,,,
256.0,0.264984,0.141791,0.084641,0.091303,0.125595
1024.0,0.352087,0.188218,0.111153,0.105944,0.135364
4096.0,0.407286,0.224301,0.136219,0.118210,0.140448


In [11]:
# Ratio = 1, epsilon vs duration across multiple threads
def plot_thread_vs_duration_ratio(ratio):
    rows = duckdb.sql(
        " SELECT threads, algo, AVG(duration_sec) as v FROM df"
        " WHERE "
        f" ratio={ratio} AND dataset='fb'"
        " GROUP BY threads, algo"
        " ORDER BY threads"
    ).df()
    result = rows.pivot(index='threads', values='v', columns=['algo'])
    print(ratio)
    display(result)
    os.makedirs('threads_vs_duration', exist_ok=True)
    result.to_csv(f'threads_vs_duration/ratio_{ratio}.txt')
plot_thread_vs_duration_ratio(10)
plot_thread_vs_duration_ratio(100)
plot_thread_vs_duration_ratio(1000)
plot_thread_vs_duration_ratio(1)

10


algo,hash_join,inlj_btree256,inlj_sampledflatpgm256,lsj_btree256,lsj_sampledflatpgm256,sort_join
threads,,,,,,
1,47.259717,6.715505,4.808543,5.186520,3.272983,3.884844
2,24.765308,3.846768,3.319870,3.229327,2.064571,2.605233
4,12.081015,2.538997,1.871465,2.076746,1.086257,1.564231
8,6.675369,1.689745,1.254484,1.384631,0.621618,1.070960
16,3.649402,1.087927,1.121795,0.871988,0.348913,0.796304


100


algo,hash_join,inlj_btree256,inlj_sampledflatpgm256,lsj_btree256,lsj_sampledflatpgm256,sort_join
threads,,,,,,
1,33.330341,1.354677,1.429695,1.236245,0.870979,3.131840
2,17.263721,0.859212,0.764755,0.949227,0.544980,1.847075
4,8.271893,0.547286,0.471367,0.356459,0.245511,0.879462
8,4.519754,0.355153,0.406387,0.378356,0.193363,0.491787
16,2.480878,0.348173,0.418114,0.219315,0.185621,0.382747


1000


algo,hash_join,inlj_btree256,inlj_sampledflatpgm256,lsj_btree256,lsj_sampledflatpgm256,sort_join
threads,,,,,,
1,13.025334,0.284472,0.338612,0.217058,0.288988,2.563320
2,7.068931,0.167652,0.210054,0.165454,0.149614,1.847984
4,3.802508,0.128468,0.160614,0.098762,0.085551,0.938329
8,2.022736,0.115407,0.119511,0.213544,0.121266,0.524207
16,1.443665,0.165546,0.188764,0.102354,0.118276,0.319549


1


algo,hash_join,inlj_btree256,inlj_sampledflatpgm256,lsj_btree256,lsj_sampledflatpgm256,sort_join
threads,,,,,,
1,76.800647,58.420543,34.154064,40.968796,21.339946,11.095250
2,49.246093,32.704744,20.591028,24.273294,13.119371,8.865352
4,25.262623,17.746799,11.918770,13.296721,6.934015,5.798322
8,14.309272,11.296346,8.252280,8.113987,3.625109,4.584295
16,8.469287,7.855749,6.045090,6.156285,2.229635,4.084857


In [8]:
# Ratio = 1, epsilon vs duration across multiple threads
def plot_duration_vs_ratio_string(threads):
    rows = duckdb.sql(
        " SELECT ratio, dataset, algo, AVG(duration_sec) as v FROM df"
        " WHERE (algo='hash_join' OR algo='sort_join' OR algo='inlj_btree256' OR algo='lsj_flatpgm256') AND"
        f" threads={threads} AND dataset='string_keys'"
        " GROUP BY ratio, algo, dataset"
        " ORDER BY ratio"
    ).df()
    result = rows.pivot(index='ratio', values='v', columns=['algo'])
    os.makedirs('duration_vs_ratio_string', exist_ok=True)
    result.to_csv(f'duration_vs_ratio_string/string.txt')
    return result
plot_duration_vs_ratio_string(1)

algo,hash_join,inlj_btree256,lsj_flatpgm256,sort_join
ratio,,,,
1,36.336855,52.538703,11.664905,6.576802
10,35.526652,9.852073,2.414677,2.125464
100,21.228238,1.313076,0.509996,1.131219
1000,10.002874,0.237509,0.172064,1.004427
